In [1]:
from pprint import pprint
import re
from collections import Counter
from copy import deepcopy
import itertools

In [2]:
with open("./stored_authors/author_url_pairings.txt", encoding="utf8") as pairings_file:
    author_url_pairings = eval(pairings_file.read())
with open("./stored_authors/unknown_authors.txt") as unknowns_file:
    unknown_authors = eval(unknowns_file.read())
with open("./stored_authors/duplicate_authors.txt") as duplicates_file:
    duplicate_authors = eval(duplicates_file.read())

In [3]:
# Used for reference when checking for authors in duplicate list

unpacked_duplicates = set()
for author_list in duplicate_authors.values():
    for author in author_list:
        unpacked_duplicates.add(author)
        
sorted_unpacked_duplicates = sorted(list(unpacked_duplicates))

In [4]:
h_likely = [] # Highly likely, unknowns that have matches
possible = [] # Moderately likely, the ones whose possible matches have similar characters
unlikely = [] # Unlikely, few matching characters

unknown_pairing = {}
for unknown in unknown_authors:
    unknown_pairing[unknown] = None

for author, pair in unknown_authors.items():

    if author in author_url_pairings:
        author_url = author_url_pairings[author]
    else:
        author_url = "no"

#     print("-----new iteration-----")
#     print(author)
#     print(author_url) # 
#     print("--end of author url--")

    found = "Unsure"
    for pair_id in pair:
        pair_url = author_url_pairings[pair_id]

#         print(pair_url) #

        if "contributions" in author_url and "contributions" in pair_url:
            author_num = re.search(r"\d+", author_url).group(0)
            pair_num = re.search(r"\d+", pair_url).group(0)
            if author_num == pair_num:
                found = pair_id
                h_likely.append([author, pair_id])

#                 print("yes!") #

        elif author_url ==  pair_url:
            found = pair_id
            h_likely.append([author, pair_id])

#             print("yes!") #

    unknown_pairing[author] = found

In [5]:
# Pairs duplicates
duplicate_pairing = {}
for author, url in author_url_pairings.items():
    if author in sorted_unpacked_duplicates:
        duplicate_pairing[url] = []
        
# Iterates through duplicates and appends pairings for each URL
for author, pair in duplicate_authors.items():
    for pair_id in pair:
        duplicate_pairing[author_url_pairings[pair_id]].append(pair_id)
        
merged_pairing = deepcopy(duplicate_pairing)

# Generates list of all the url IDs for any urls that have them (i.e. urls with "contributions"
url_num_list = []
for url in duplicate_pairing:
    if "contributions" in url:
        url_num_list.append(re.search(r"\d+", url).group(0))

# Generates list for all url IDs that have > 1 urls that contain them
same_num_list = [num for num, count in Counter(url_num_list).items() if count > 1]

# Iterates through nums then urls: longest_url -> longest (most complete name) to set as the new merged key
# merged_entries -> merging of the values of the keys for assigning to the new merged key 
for num in same_num_list:
    longest_url = ""        # Will be used as the key of the new merged entry (longer name is almost always the most detailed name)
    url_list = []           # Keeps track of urls to remove them from merged_pairing 
    merged_entries = []     # Used a value for new merged entry
    
    # Finds values and key for merging values
    for url in duplicate_pairing:
        if num in url:
            url_list.append(url)
            merged_entries += duplicate_pairing[url]
            
            # Assigns longest_url for use as a key
            if len(url) > len(longest_url):
                longest_url = url
    
    # Merges/assigns values import itertools
    for url in url_list:
        if url == longest_url:
            merged_pairing[url] = merged_entries
            
        # Removes other unmerged entries
        else:
            merged_pairing.pop(url)

for url, matches in merged_pairing.items():
    if len(matches) > 1:
        h_likely.append(matches)

In [6]:
# for list_thing in h_likely:
#     if len(list_thing) == 2:
#         print(list_thing)

all_h_likely_values = list(itertools.chain(*h_likely))
# pprint(h_first)
h_likely_duplicates = [author for author, count in Counter(all_h_likely_values).items() if count > 1]
# pprint(h_likely_duplicates)
print(len(h_likely_duplicates))
print(len(h_likely))

28
197


In [15]:
unknown_total = len(unknown_pairing)
print("Total: " + str(unknown_total))

unknown_not_unsure = len([author for author, status in unknown_pairing.items() if status != "Unsure"])
print("Not unsure: " + str(unknown_not_unsure))

unknown_unsure = total - not_unsure
print("Unsure: " + str(unknown_unsure))


Total: 245
Not unsure: 151
Unsure: 94


In [21]:
duplicate_total = len(merged_pairing)
print("Total: " + str(duplicate_total))

duplicate_matches = len([url for url, matches in merged_pairing.items() if len(matches) > 1])
print("Matches: " + str(duplicate_matches))

duplicate_individual = duplicate_total - duplicate_matches
print("Individuals (unsure): " + str(duplicate_individual))

Total: 103
Matches: 36
Individuals (unsure): 67


In [24]:
overall_total = unknown_total + duplicate_total
print("Overall Total: " + str(overall_total)) 

overall_highly_likely = len(h_likely)
print("Overall highly_likely: " + str(overall_highly_likely))

overall_unsure = overall_total - overall_highly_likely
print("Overall unsure: " + str(overall_unsure))

Overall Total: 348
Overall highly_likely: 197
Overall unsure: 151


In [26]:
# Selects an author as the "root" (the longest/one with the most special characters)
processing_roots = {}

# Iterates through lists in h_likely
for matches in h_likely:
    root_name = ""                    # Longest/most accurate name to have everything merge into for database
    to_be_merged = deepcopy(matches)
    
    for author in matches: 
        print("placeholder") # do something here, look at first/middle/last name, maybe rework authors_ids to include orc_id?

In [27]:
pprint(h_likely)

[['0000-0002-6496-8411', 'b7c2c5fbb917762d934fdaec1a6c4f7272ea3254'],
 ['020a08bc0da60eefe9886f8cf54a14db1f1a43ab',
  'c4aa2acfa847b763f573e4c50bc5c24418fd59d3'],
 ['02470cc05fe10488eae25b981d88bba34b3c80be',
  '966bbe37507c538696f134495ca6d67b360b64a2'],
 ['036e372dcae3263b44e3566b0afb897c09843add',
  '624566d921f2ced905a2803e3b30a08a990ddafc'],
 ['04738a3e4ecb3c4b065063280d614143956e7331',
  '52b9ec426c110236605026615b0db869a094b255'],
 ['04b68c3604022473f9658d2bf55a583665efd86e', '0000-0002-2980-7997'],
 ['096a326cd38289ea0bf9b8623ba57dff97d3cd80',
  '92d16c28f557037ca62ba48feae99f6c86820fbc'],
 ['09e88eba05218f20aa99885a2ca16571a999dc74',
  'b2106bdb4c8425c5650d24f679b3cd98b3c6b4a0'],
 ['0adde808f11b05b60072e8b6995c6ac15474b661',
  '25651571ae5c23818e37b1339631fee60c067743'],
 ['0c445d4eb226142eb7e436fd1434baaad7a075d1',
  '0016033f47b68850ca16517353b7967896496108'],
 ['0d0ed8c73170487868df86c11d2fabdcc3cb86e0',
  '78e6eb919e5501ee8e9c434b3e0abda8af8bc7fc'],
 ['0d853b61e5fab5c30796

In [7]:
pprint(merged_pairing)

{'https://www.researchgate.net/profile/Alirio_Rodrigues': ['04a46020814d276f91dbe2ddbe804d96c7d7c7b9',
                                                           '584647c9748417e6dddb743141ae895a61de3dce'],
 'https://www.researchgate.net/profile/Ana_Ribeiro25': ['831da78505f36a7f7f27627e3cf7c7ce70f3c814',
                                                        '8b80ee24d983f356b3117484ac1a6f9a2e7ce441'],
 'https://www.researchgate.net/profile/Asim_Bhaumik': ['0000-0002-4907-7418',
                                                       'c8579580ef35af4669ea16470ba8f628a7b7b1c0'],
 'https://www.researchgate.net/profile/Badie_Morsi': ['8602215936c1eb9465b58b12bc05093ff26c2e90',
                                                      'f2d062d5c2d48c419af09fd0b0e4ffe169d28306'],
 'https://www.researchgate.net/profile/Bidyut_Saha2': ['4ff206651a890dafc062f9f7fa51888d77b2bca3',
                                                       '975de28b0249c30fde13e4d1214d21d5d4ac6763'],
 'https://www.rese